In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import pickle
from matplotlib import pyplot as plt
import statsmodels.api as sm
from matplotlib.ticker import Formatter
import collections
from decimal import Decimal, getcontext, ROUND_HALF_UP

logSH = pd.read_csv('/mnt/data/20201208/zt_6802/zt_6802_20201208_168.prevde/Logs_full/mdLog_SH_20201208_0904.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]

logSH1 = pd.read_csv('/mnt/data/20201208/zt_6802/zt_6802_20201208_168.30.6.171/Logs_full/mdLog_SH_20201208_0903.csv',
                    encoding="utf-8").loc[:, ["clockAtArrival", "sequenceNo", "source", "StockID",
                                               "exchange", "time", "cum_volume", "cum_amount", "close",
                                               "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
                                               "bid2q", "bid3q", "bid4q", "bid5q", "ask1p", "ask2p",
                                               "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
                                               "ask4q", "ask5q", "openPrice", "numTrades"]]

# # 1. data format
print("original data source type: ")
print(logSH["source"].unique())
print("new data source type: ")
print(logSH1["source"].unique())
print(logSH["time"].unique())
print(logSH1["time"].unique())

original data source type: 
[23 22]
new data source type: 
[31]
['09:04:19.000' '09:04:20.360' '09:04:21.070' ... '15:32:32.510'
 '15:32:32.000' '15:32:33.250']
['09:03:26.000' '09:03:28.000' '09:03:31.000' ... '15:31:26.000'
 '15:31:27.000' '15:31:29.000']


In [2]:
logSH["time"] = logSH["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))
logSH1["time"] = logSH1["time"].apply(lambda x: int((x.replace(':', "")).replace(".", "")))

display(logSH[(logSH["StockID"] == 600004) & (logSH["time"] >= 143000000) & (logSH["time"] <= 145000000)])
display(logSH1[(logSH1["StockID"] == 600004) & (logSH1["time"] >= 143000000) & (logSH1["time"] <= 145000000)])

,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
9793166,1607409002028280,23996681,23,600004,SH,143001000,8254846,1.225330e+08,14.72,14.72,...,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11613
9793358,1607409002387219,23997133,22,600004,SH,143002390,8254946,1.225345e+08,14.73,14.72,...,14.75,14.76,14.77,14500,3100,17200,4100,2100,15.06,11614
9795459,1607409005079669,24001597,23,600004,SH,143004000,8255346,1.225404e+08,14.73,14.72,...,14.75,14.76,14.77,14100,3100,17200,4100,2100,15.06,11615
9795630,1607409005385287,24002127,22,600004,SH,143005390,8255346,1.225404e+08,14.73,14.72,...,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11615
9797643,1607409008095709,24006606,23,600004,SH,143007000,8256946,1.225639e+08,14.72,14.72,...,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10690345,1607410193482559,26124711,22,600004,SH,144953490,9898691,1.467001e+08,14.67,14.67,...,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13869
10692056,1607410195743530,26128811,23,600004,SH,144955000,9898691,1.467001e+08,14.67,14.67,...,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13869
10692761,1607410196480707,26130544,22,600004,SH,144956490,9898691,1.467001e+08,14.67,14.67,...,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13869
10694445,1607410198781689,26134613,23,600004,SH,144958000,9899591,1.467133e+08,14.67,14.67,...,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13873


,clockAtArrival,sequenceNo,source,StockID,exchange,time,cum_volume,cum_amount,close,bid1p,...,ask3p,ask4p,ask5p,ask1q,ask2q,ask3q,ask4q,ask5q,openPrice,numTrades
4828225,1607409002032559,88752799,31,600004,SH,143001000,8254846,122532999.0,14.72,14.72,...,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11613
4829340,1607409005082675,88770493,31,600004,SH,143004000,8255346,122540364.0,14.73,14.72,...,14.75,14.76,14.77,14100,3100,17200,4100,2100,15.06,11615
4830458,1607409008100110,88789461,31,600004,SH,143007000,8256946,122563916.0,14.72,14.72,...,14.75,14.76,14.77,14600,3100,17200,4100,2100,15.06,11619
4831390,1607409010828221,88807949,31,600004,SH,143010000,8257046,122565388.0,14.72,14.72,...,14.75,14.76,14.77,14600,2300,17200,4100,2100,15.06,11620
4832503,1607409013817315,88826963,31,600004,SH,143013000,8257046,122565388.0,14.72,14.72,...,14.75,14.76,14.77,14600,2300,17200,4100,2100,15.06,11620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5272846,1607410186768515,96233318,31,600004,SH,144946000,9893091,146617918.0,14.67,14.67,...,14.70,14.71,14.72,30500,19800,22000,11700,10200,15.06,13862
5274038,1607410189793305,96254701,31,600004,SH,144949000,9895191,146648745.0,14.67,14.67,...,14.70,14.71,14.72,30000,19200,22000,11700,10200,15.06,13864
5275227,1607410192782955,96276263,31,600004,SH,144952000,9896691,146670751.0,14.67,14.67,...,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13867
5276413,1607410195747937,96296616,31,600004,SH,144955000,9898691,146700091.0,14.67,14.67,...,14.70,14.71,14.72,29900,19200,22000,11700,10200,15.06,13869


In [4]:
# in_dex = logSH[logSH["source"] == 5]["StockID"].unique()
in_dex = [16, 300, 852, 905]
data1 = logSH[~logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0) & (logSH['source'] == 23)]
data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1['cum_volume'] > 0) & (logSH1['source'] == 31)]

columns = ["StockID", "cum_volume", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q",
           "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
           "ask4q", "ask5q", 'numTrades', 'openPrice']
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()
n1 = len(data1_1["StockID"].unique())
n2 = len(data2_1["StockID"].unique())

print(n1)
print(n2)
if n1 != n2:
    sl = list(set(data1_1["StockID"].unique()) & (set(data2_1["StockID"].unique())))
    data1_1 = data1_1[data1_1["StockID"].isin(sl)]
    data2_1 = data2_1[data2_1["StockID"].isin(sl)]
for cols in ['cum_amount']:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)
data1_1.loc[data1_1['cum_volume'] > 0, 'openPrice'] = data1_1[data1_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
data2_1.loc[data2_1['cum_volume'] > 0, 'openPrice'] = data2_1[data2_1['cum_volume'] > 0].groupby('StockID')['openPrice'].transform('max')
test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])
    print(len(test[np.isnan(test["sequenceNo_y"])])/n1)
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(test[np.isnan(test["sequenceNo_x"])])
    display(n2-n1)
    print((n2-n1)/n1)

1457
1459
4825891
4809600
4825892
-----------------------------------------------


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,...,ask5q,openPrice,numTrades,index_y,clockAtArrival_y,sequenceNo_y,source_y,exchange_y,time_y,cum_amount_y
6112,239265.0,1.607391e+15,334831.0,23.0,600683,SH,93000000.0,400,1.736000e+03,4.34,...,20600,4.34,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6209,239579.0,1.607391e+15,340733.0,23.0,600192,SH,93000000.0,1400,7.154000e+03,5.11,...,11200,5.11,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6211,239582.0,1.607391e+15,340736.0,23.0,603908,SH,93000000.0,500,1.380500e+04,27.64,...,1100,27.59,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6212,239583.0,1.607391e+15,340737.0,23.0,603110,SH,93000000.0,600,1.167000e+04,19.46,...,100,19.44,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7607,242614.0,1.607391e+15,365000.0,23.0,600292,SH,93003000.0,3100,1.726600e+04,5.56,...,500,5.57,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824397,11163396.0,1.607411e+15,27245591.0,23.0,603700,SH,145959000.0,2911959,8.293739e+07,28.68,...,0,28.24,5373,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4824417,11163418.0,1.607411e+15,27245613.0,23.0,600089,SH,145959000.0,25713865,2.123109e+08,8.29,...,0,8.24,15268,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4824421,11163422.0,1.607411e+15,27245617.0,23.0,600185,SH,145959000.0,31303321,2.543787e+08,8.11,...,0,8.18,27696,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4824429,11163430.0,1.607411e+15,27245625.0,23.0,600406,SH,145959000.0,11210102,2.533782e+08,22.61,...,0,22.43,23966,NaN,NaN,NaN,NaN,NaN,NaN,NaN


0.0033759568958354014


In [5]:
# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

3762.0
8041.0
13226.0
2581.0
3592.0
6741.5
0.10896812811213576
0.8910220997634106


In [7]:
in_dex = [16, 300, 852, 905]
data1 = logSH[logSH["StockID"].isin(in_dex) & (logSH["cum_volume"] > 0)]
data2 = logSH1[logSH1["StockID"].isin(in_dex) & (logSH1["cum_volume"] > 0)]
# data2 = data2[data2["openPrice"] != 0]
columns = ["StockID", "cum_volume", "close", "openPrice"]
data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

for cols in ["cum_amount"]:
    data1_1[cols] = data1_1[cols].round(2)
    data2_1[cols] = data2_1[cols].round(2)

test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
n1 = test["sequenceNo_x"].count()
n2 = test["sequenceNo_y"].count()
len1 = len(test)
print(n1)
print(n2)
print(len1)
if n2 < len1:
    display("test is not complete:")
    display(test[np.isnan(test["sequenceNo_y"])])

11400
11241
11400


'test is not complete:'

,index_x,clockAtArrival_x,sequenceNo_x,source_x,StockID,exchange_x,time_x,cum_volume,cum_amount_x,close,...,ask2p_y,ask3p_y,ask4p_y,ask5p_y,ask1q_y,ask2q_y,ask3q_y,ask4q_y,ask5q_y,numTrades_y
46,287781,1607391051209611,670654,22,852,SH,93051190,4067596,4.138115e+09,6734.4294,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,335109,1607391101240872,883773,22,852,SH,93141220,5604621,5.745572e+09,6735.2670,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,343912,1607391111180978,919602,22,852,SH,93151160,5853942,6.018171e+09,6735.3318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,399657,1607391171153093,1150535,22,852,SH,93251130,7433379,7.719241e+09,6734.4912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238,509724,1607391291207000,1597707,22,852,SH,93451190,10560159,1.091498e+10,6745.0136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10802,10451048,1607409891215896,25522533,22,852,SH,144451220,109275702,1.275901e+11,6737.7873,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10850,10498427,1607409951220261,25643950,22,852,SH,144551220,109805705,1.282206e+11,6739.1442,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10994,10640129,1607410131239772,25997956,22,852,SH,144851220,111550220,1.301009e+11,6736.0772,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11042,10688980,1607410191250897,26120868,22,852,SH,144951190,112246197,1.308340e+11,6734.5669,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# speed check
test1 = test[~np.isnan(test["sequenceNo_x"])]
test1 = test1[~np.isnan(test1["sequenceNo_y"])]
diff = test1["clockAtArrival_y"] - test1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

18304.5
42202.0
76599.5
8309.0
11990.0
15937.0
0.17783115381193845
0.8220798861311271


In [9]:
import pandas as pd
SH = pd.read_csv(r'/mnt/data/20201208/zt_6802/zt_6802_20201208_168.prevde/Logs_full/mdTradeLog_20201208_0904.csv',
                   encoding="utf-8", index_col=False)
SH1 = pd.read_csv(r'/mnt/data/20201208/zt_6802/zt_6802_20201208_168.30.6.171/Logs_full/mdTradeLog_20201208_0903.csv',
                   encoding="utf-8", index_col=False)
SH = SH[SH["exchId"] == 1]
SH1 = SH1[SH1["exchId"] == 1]

In [10]:
import numpy as np
SH["ExecType"] = SH["ExecType"].apply(lambda x: str(x))
SH1["ExecType"] = SH1["ExecType"].apply(lambda x: str(x))

columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag","ExecType",
           "BidApplSeqNum", "OfferApplSeqNum", 'secid', 'exchId']

n1 = len(SH["SecurityID"].unique())
n2 = len(SH1["SecurityID"].unique())
print(n1)
print(n2)
print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))
print(set(SH1["SecurityID"].unique()) - set(SH["SecurityID"].unique()))
if n1 != n2:
    sl = list(set(SH["SecurityID"].unique()) & set(SH1["SecurityID"].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
re = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
n1 = re["sequenceNo_x"].count()
n2 = re["sequenceNo_y"].count()
len1 = len(re)
print(n1)
print(n2)
print(len1)
print("-----------------------------------------------")
if n2 < len1:
    display("test is not complete:")
    display(re[np.isnan(re["sequenceNo_y"])])
    print(len(re[np.isnan(re["sequenceNo_y"])]))
    print(np.sort(re[np.isnan(re["sequenceNo_y"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_y"])]["SecurityID"].unique())
if (len1 == n2) & (n1 < len1):
    display("baseline is not complete:")
    display(re[np.isnan(re["sequenceNo_x"])])
    print(np.sort(re[np.isnan(re["sequenceNo_x"])]["TransactTime"].unique()))
    print(len(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique()))
    print(re[np.isnan(re["sequenceNo_x"])]["SecurityID"].unique())
    display(n2-n1)

1457
1456
{511880}
set()
16192614
16192614
16192614
-----------------------------------------------


In [11]:
# speed check
re1 = re[~np.isnan(re["sequenceNo_x"])]
re1 = re1[~np.isnan(re1["sequenceNo_y"])]
diff = re1["clockAtArrival_y"] - re1["clockAtArrival_x"]
print(np.quantile(abs(diff[diff < 0]), 0.25))
print(np.median(abs(diff[diff < 0])))
print(np.quantile(abs(diff[diff < 0]), 0.75))
print(np.quantile(diff[diff > 0], 0.25))
print(np.median(diff[diff > 0]))
print(np.quantile(diff[diff > 0], 0.75))
print(sum(diff < 0)/len(diff))
print(sum(diff > 0)/len(diff))

2930.0
6445.0
12045.0
6031.0
8103.0
11785.0
0.059597665948190943
0.9403965906925219


In [24]:
stocks = SH["SecurityID"].unique()
m = 0
diff = []
for i in stocks:
    re1 = pd.merge(SH[SH["SecurityID"] == i], SH1[SH1["SecurityID"] == i], left_on=columns, right_on=columns, how="inner")
    if (sorted(re1["sequenceNo_y"]) != re1["sequenceNo_y"]).any() == False:
        m = m + 1
    else:
        diff.append(i)
if m == len(stocks):
    print("SHE Trade Data have same ordering within stocks")
else:
    print("SHE Trade Data don't have same ordering within stocks")

SHE Trade Data have same ordering within stocks
